In [ ]:
# for jupyter notebooks, see: 
# https://digital-science.github.io/dimcli/getting-started.html#dimcli-with-jupyter-notebooks
# https://api-lab.dimensions.ai/cookbooks/1-getting-started/4-Dimcli-magic-commands.html

# for working with dataframes: https://github.com/digital-science/dimcli/blob/master/examples/3-Working-with-dataframes.ipynb

In [5]:
import os
import dimcli
import dimcli.utils as du
import pandas as pd
import numpy as np
import query

# Dimensions Login & DSL
dimcli.login(key="02CB1C5A52B5430AA173E8D88F59A986",  
                 endpoint="https://app.dimensions.ai/api/dsl/v2")
dsl = dimcli.Dsl()

Dimcli - Dimensions API Client (v0.9.9.1)
Connected to: <https://app.dimensions.ai/api/dsl/v2> - DSL v2.6
Method: manual login
====
Heads up! The latest Dimcli version is  1.0.2
You have installed:  0.9.9.1
====
Please upgrade: `pip install dimcli -U`


In [6]:
df = pd.read_csv("/Users/ading/Downloads/csv_refs_v2/1970_AIR_QUALITY_CRITERIA_FOR_PHOTOCHEMICAL_OXIDANTS_ocr_OCR.csv")
df_titles_years = df[["Title", "Author", "Date"]]

subset_df = df_titles_years[0:10]
subset_df


,Title,Author,Date
0,CPEHS r National Air Pollution Control Adminis...,Air Quality Criteria for Sulfur Oxides. U.S. D...,January 1969.
1,Ozone Chemistry and Technology.,,
2,Some Analyses for PAN and Studies of its Struc...,"Nicksic, S.W., J. Harkins, and P.K. Mueller.","J:11-18,January 1967."
3,Absorptivities for Infrared Determination of P...,"Stephens, E.R.",April 1964.
4,Photochemical Reaction Products in Air Pollution.,"Stephens, E.R. et al.",June 1961.
5,The Photochemical Olefin-Nitrogen Oxides React...,"Stephens, E.R.",1961.
6,Recent Developments in the Study of the Organi...,"Stephens, E.R. ct al.",November 1956.
7,Photochemistry of Air Pollution.,"Leighton, P.A.",1961.
8,Synergistic Effects in the Photooxidation of M...,"Bufalini, J.J. and A.P. Altshuller.",February 1967.
9,Addition of Atoms to Olefins in the Gas Phase.,"Cvetanovit, R.J.",1963.


In [7]:
query_result_array = []
positive_results = pd.DataFrame()
negative_results = pd.DataFrame()
dateless = pd.DataFrame()
bad_format = pd.DataFrame()

for index, [title, author, date] in subset_df.iterrows():
    # remove colons/brackets    
    title = query.format_title(title)
    # format YYYY
    print(date)
    year = query.extract_year(date)
    print(year)
    # get first author last name
    first_author = query.get_first_author(author)
    
    query_res = query.query_fuzzy_title_year_author(title, year, author)
    query_df = dsl.query(query_res).as_dataframe()

    # Separate into succesful/unsuccesful references
    if query_df is None:
        negative_results = pd.concat([negative_results, df.iloc[index].to_frame().T], axis=0)
    elif len(query_df) == 0:
        negative_results = pd.concat([negative_results, df.iloc[index].to_frame().T], axis=0)
    else:
        query_result_array.append(query_df)

if len(query_result_array) != 0:
    positive_results = pd.concat(query_result_array).drop_duplicates(subset='id')

positive_results

January 1969.
1969
Returned Errors: 1
Time: 0.83s
Query Error
4 QuerySyntaxErrors found

3 LexerErrors found
  * [Line 5:68] ('U.S. DHE W, PHS,') token recognition error at: 'U.'
  * [Line 5:70] ('S. DHE W, PHS,') token recognition error at: 'S.'
  * [Line 5:77] ('W, PHS,') token recognition error at: 'W,'

1 ParserError found
  * [Line 5:12] ('where') mismatched input 'where' expecting <EOF>
 


AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
# Subset query with title filter

query_results = []

for index, element in subset_df.iterrows():

    title = element[0]
    year = int(element[1][-5:-1])

    query = f"""
            search publications in title_only 
                for "{du.dsl_escape(title)}" 
            return publications limit 10
            """

    df = dsl.query(query).as_dataframe()
    query_results.append(df)

df = pd.concat(query_results).\
   drop_duplicates(subset='id')

In [ ]:
df

In [ ]:
# Full query (94 references) with title, year filters

query_results = []

for index, element in df_titles_years.iterrows():

    title = element[0]
    year = int(element[1][-5:-1])

    query = f"""
            search publications in title_only 
                for "{dsl_escape(title)}" 
                where year = {year}
            return publications limit 10
            """

    df = dsl.query(query).as_dataframe()
    query_results.append(df)

df = pd.concat(query_results).\
   drop_duplicates(subset='id')

# Note that the resulting df has some elements with type = "chapter" instead of "article"
# Also note that some titles don't make sense

In [ ]:
df